In [1]:
import os
import librosa
import math
import json
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/dataset"
JSON_PATH = "data.json"
SAMPLE_RATE = 22050
DURATION = 4 #in secondi
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

In [ ]:
def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
  data = {
      "mapping": [], # i nomi degli strumenti diventono indici del vettore
      "labels": [], # indica gli indici corrispondenti alla mfcc
      "mfcc": [] # sono gli ingressi
  }
  samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
  expected_num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length) # i numeri decimali sono portati al numero intero più prossimo
  for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    if dirpath is not dataset_path:
      dirpath_components = os.path.split(dirpath) # strumento/tastiera => ["nstrumento, "tastiera"]
      semantic_label = dirpath_components[-1] # nome della cartella
      data["mapping"].append(semantic_label)
      print("\nProcessando {}".format(semantic_label))
      for f in filenames:
        file_path = os.path.join(dirpath, f)
        signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)  # carica il file audio
        for s in range (num_segments): # processo i segmenti e conservo il dato
          start = samples_per_segment * s
          finish = start + samples_per_segment

          mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=sr, n_fft=n_fft, n_mfcc=n_mfcc, hop_length=hop_length)
          mfcc = mfcc.T
          if len(mfcc) == expected_num_mfcc_vectors_per_segment:
            data["mfcc"].append(mfcc.tolist())
            data["labels"].append(i-1)
            print("{}, segment:{}".format(file_path, s))
    with open(json_path, "w") as fp:
      json.dump(data, fp, indent=4)

if __name__=="__main__":
  save_mfcc(DATASET_PATH, JSON_PATH, num_segments=5)

Output streaming troncato alle ultime 5000 righe.
/content/drive/MyDrive/Colab Notebooks/dataset/Guitar Electronic/guitar_electronic_022-037-050.wav, segment:0
/content/drive/MyDrive/Colab Notebooks/dataset/Guitar Electronic/guitar_electronic_022-037-050.wav, segment:1
/content/drive/MyDrive/Colab Notebooks/dataset/Guitar Electronic/guitar_electronic_022-037-050.wav, segment:2
/content/drive/MyDrive/Colab Notebooks/dataset/Guitar Electronic/guitar_electronic_022-037-050.wav, segment:3
/content/drive/MyDrive/Colab Notebooks/dataset/Guitar Electronic/guitar_electronic_022-037-050.wav, segment:4
/content/drive/MyDrive/Colab Notebooks/dataset/Guitar Electronic/guitar_electronic_028-075-127.wav, segment:0
/content/drive/MyDrive/Colab Notebooks/dataset/Guitar Electronic/guitar_electronic_028-075-127.wav, segment:1
/content/drive/MyDrive/Colab Notebooks/dataset/Guitar Electronic/guitar_electronic_028-075-127.wav, segment:2
/content/drive/MyDrive/Colab Notebooks/dataset/Guitar Electronic/guita

Scorre tra tutte le note musicali dei vari strumenti e ne processa una alla volta in modo specifico